In [1]:
import csv
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url_xls = "https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide"
url_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
url_countries_csv = "https://raw.githubusercontent.com/dbouquin/IS_608/master/NanosatDB_munging/Countries-Continents.csv"

In [3]:
page = requests.get(url_xls).text

In [4]:
soup = BeautifulSoup(page, 'html.parser')

In [5]:
data_attribute = soup.find_all("a", attrs={"data-toggle": "tooltip"})

In [6]:
for link in data_attribute:
    data_link = link.get('href')

In [7]:
with requests.Session() as s:
    download = s.get(data_link).content
    dataframe = pd.read_excel(download)

In [8]:
dataframe.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
0,2020-03-23,23,3,2020,10,0,Afghanistan,AF
1,2020-03-22,22,3,2020,0,0,Afghanistan,AF
2,2020-03-21,21,3,2020,2,0,Afghanistan,AF
3,2020-03-20,20,3,2020,0,0,Afghanistan,AF
4,2020-03-19,19,3,2020,0,0,Afghanistan,AF


In [9]:
dataframe.groupby("Countries and territories").mean().head()

,Day,Month,Year,Cases,Deaths
Countries and territories,,,,,
Afghanistan,15.608108,1.891892,2019.986486,0.459459,0.000000
Albania,16.000000,3.000000,2020.000000,5.933333,0.133333
Algeria,15.177215,1.962025,2019.987342,1.291139,0.189873
Andorra,17.300000,3.000000,2020.000000,11.300000,0.000000
Angola,22.500000,3.000000,2020.000000,1.000000,0.000000


In [10]:
df_github = pd.read_csv(url_csv)

In [11]:
data_by_country = df_github.groupby("Country/Region").mean()

In [12]:
df_github.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,75,82,114,147,177,212,272,322,411,599
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,701,773,839,825,878,889,924,963,1007,1086
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,200,212,226,243,266,313,345,385,432,455
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,2
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,197,238,428,566,673,790,900,1030,1183,1306


In [13]:
df_github_countries = df_github.iloc[:, 0:4]

In [14]:
df_github_countries

,Province/State,Country/Region,Lat,Long
0,NaN,Thailand,15.0000,101.0000
1,NaN,Japan,36.0000,138.0000
2,NaN,Singapore,1.2833,103.8333
3,NaN,Nepal,28.1667,84.2500
4,NaN,Malaysia,2.5000,112.5000
...,...,...,...,...
482,NaN,Dominica,15.4150,-61.3710
483,NaN,Grenada,12.1165,-61.6790
484,NaN,Mozambique,-18.6657,35.5296
485,NaN,Syria,34.8021,38.9968


In [15]:
df_github_countries.iloc[0,0]

nan

In [53]:
country_df = df_github_countries.iloc[:, 1:4].drop_duplicates(subset=['Country/Region'],keep='last',ignore_index=False)

In [54]:
len(country_df)

171

In [23]:
country_df.to_csv("country.csv")

In [24]:
continents = ["Africa", "Antarctica", "Asia", "Europe", "North America", "Oceania", "South America", "None"]

In [25]:
continents_df = pd.DataFrame(continents)

In [26]:
continents_df.to_csv("continent.csv")

In [27]:
prov_state = df_github.iloc[:, 0].drop_duplicates(keep='first')

In [28]:
prov_state = prov_state.reset_index()

In [29]:
prov_state = prov_state.drop(["index"],axis=1)

In [30]:
prov_state.iloc[0, :] = "No province/state"

In [31]:
prov_state

,Province/State
0,No province/state
1,British Columbia
2,New South Wales
3,Victoria
4,Queensland
...,...
319,New Caledonia
320,Bermuda
321,Sint Maarten
322,Isle of Man


In [32]:
prov_state.to_csv("province_state.csv")

In [33]:
import math
region_codes = []
for region in df_github_countries.iloc[:,0]:
    if isinstance(region, str):
        pass
    elif np.isnan(region):
        region_codes.append(322)
        continue
    for i, item in enumerate(prov_state.iloc[:,0]):
        if item == region:
            region_codes.append(i)

print(len(region_codes))

487


In [34]:
df_github_countries["region_codes"] = region_codes

In [35]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes
0,NaN,Thailand,15.0000,101.0000,322
1,NaN,Japan,36.0000,138.0000,322
2,NaN,Singapore,1.2833,103.8333,322
3,NaN,Nepal,28.1667,84.2500,322
4,NaN,Malaysia,2.5000,112.5000,322
...,...,...,...,...,...
482,NaN,Dominica,15.4150,-61.3710,322
483,NaN,Grenada,12.1165,-61.6790,322
484,NaN,Mozambique,-18.6657,35.5296,322
485,NaN,Syria,34.8021,38.9968,322


In [36]:
df_countries_continents = pd.read_csv(url_countries_csv)

In [37]:
df_countries_continents

,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina
...,...,...
189,South America,Paraguay
190,South America,Peru
191,South America,Suriname
192,South America,Uruguay


In [38]:
continent_codes = []
for item in df_github_countries.iloc[:,1]:
    old_size = len(continent_codes)
    for country in df_countries_continents.iterrows():
        if item==country[1][1]:
            continent_codes.append(country[1][0])
            
    new_size = len(continent_codes)
    if old_size==new_size:
        continent_codes.append("NA")

In [39]:
df_github_countries['continent'] = continent_codes

In [40]:
df_github_countries.head()

,Province/State,Country/Region,Lat,Long,region_codes,continent
0,NaN,Thailand,15.0000,101.0000,322,Asia
1,NaN,Japan,36.0000,138.0000,322,Asia
2,NaN,Singapore,1.2833,103.8333,322,Asia
3,NaN,Nepal,28.1667,84.2500,322,Asia
4,NaN,Malaysia,2.5000,112.5000,322,Asia


In [41]:
df_github_countries.iloc[1,1]

'Japan'

In [44]:
for i, line in enumerate(df_github_countries.iterrows()):
    if line [1][1]=="North Macedonia":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Martinique":
        df_github_countries.iloc[i,5]="North America"
    elif line[1][1]=="Burkina Faso":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Holy See":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Cruise Ship":
        df_github_countries.iloc[i,5]="None"
    elif line[1][1]=="Czechia":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Taiwan*":
        df_github_countries.iloc[i,5]="Asia"
    elif line[1][1]=="Russia":
        df_github_countries.iloc[i,5]="Asia"
    elif line[1][1]=="Congo (Kinshasa)":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Cote d'Ivoire":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Eswatini":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Kosovo":
        df_github_countries.iloc[i,5]="Europe"
    elif line[1][1]=="Congo (Brazzaville)":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Gambia, The":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Bahamas, The":
        df_github_countries.iloc[i,5]="North America"
    elif line[1][1]=="Cabo Verde":
        df_github_countries.iloc[i,5]="Africa"
    elif line[1][1]=="Timor-Leste":
        df_github_countries.iloc[i,5]="Asia"

In [45]:
for i, line in enumerate(df_github_countries.iterrows()):
    if line[1][5]=="NA":
        print(line)

In [46]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent
0,NaN,Thailand,15.0000,101.0000,322,Asia
1,NaN,Japan,36.0000,138.0000,322,Asia
2,NaN,Singapore,1.2833,103.8333,322,Asia
3,NaN,Nepal,28.1667,84.2500,322,Asia
4,NaN,Malaysia,2.5000,112.5000,322,Asia
...,...,...,...,...,...,...
482,NaN,Dominica,15.4150,-61.3710,322,North America
483,NaN,Grenada,12.1165,-61.6790,322,North America
484,NaN,Mozambique,-18.6657,35.5296,322,Africa
485,NaN,Syria,34.8021,38.9968,322,Asia


In [47]:
import math
continent_codes = []
for continent in df_github_countries.iloc[:,-1]:
    if isinstance(continent, str):
        pass
    elif np.isnan(continent):
        print("WHAT")
    for i, item in enumerate(continents_df.iloc[:,0]):
        if item == continent:
            continent_codes.append(i)

print(len(continent_codes))

487


In [48]:
df_github_countries['continent_codes'] = continent_codes

In [49]:
df_github_countries

,Province/State,Country/Region,Lat,Long,region_codes,continent,continent_codes
0,NaN,Thailand,15.0000,101.0000,322,Asia,2
1,NaN,Japan,36.0000,138.0000,322,Asia,2
2,NaN,Singapore,1.2833,103.8333,322,Asia,2
3,NaN,Nepal,28.1667,84.2500,322,Asia,2
4,NaN,Malaysia,2.5000,112.5000,322,Asia,2
...,...,...,...,...,...,...,...
482,NaN,Dominica,15.4150,-61.3710,322,North America,4
483,NaN,Grenada,12.1165,-61.6790,322,North America,4
484,NaN,Mozambique,-18.6657,35.5296,322,Africa,0
485,NaN,Syria,34.8021,38.9968,322,Asia,2


In [50]:
cases = df_github_countries.drop("continent", axis=1).drop("Lat", axis=1).drop("Long", axis=1).drop("Province/State", axis=1)

In [51]:
cases

,Country/Region,region_codes,continent_codes
0,Thailand,322,2
1,Japan,322,2
2,Singapore,322,2
3,Nepal,322,2
4,Malaysia,322,2
...,...,...,...
482,Dominica,322,4
483,Grenada,322,4
484,Mozambique,322,0
485,Syria,322,2


In [55]:
import math
country_codes = []
for case in cases.iloc[:,0]:
    for i, item in enumerate(country_df.iloc[:,0]):
        if item == case:
            country_codes.append(i)
            
print(len(country_codes))

487


In [56]:
cases['country_code'] = country_codes

In [57]:
cases = cases.drop('Country/Region', axis=1)

In [58]:
cases

,region_codes,continent_codes,country_code
0,322,2,0
1,322,2,1
2,322,2,2
3,322,2,3
4,322,2,4
...,...,...,...
482,322,4,166
483,322,4,167
484,322,0,168
485,322,2,169


In [59]:
cases_full = pd.concat([cases, df_github.iloc[:,4:]], axis=1)

In [60]:
cases_full_melt = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=['1/22/20'])

In [61]:
cases_full_melt

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,1/22/20,2
1,322,2,1,1/22/20,2
2,322,2,2,1/22/20,0
3,322,2,3,1/22/20,0
4,322,2,4,1/22/20,0
...,...,...,...,...,...
482,322,4,166,1/22/20,0
483,322,4,167,1/22/20,0
484,322,0,168,1/22/20,0
485,322,2,169,1/22/20,0


In [62]:
cases_full.shape

(487, 64)

In [63]:
for i, column in enumerate(cases_full.iloc[:,4:]):
    melted_df = pd.melt(cases_full, id_vars=['region_codes', 'continent_codes', 'country_code'], value_vars=[column])
    cases_full_melt = cases_full_melt.append(melted_df)
    print(i, cases_full_melt.shape)

0 (974, 5)
1 (1461, 5)
2 (1948, 5)
3 (2435, 5)
4 (2922, 5)
5 (3409, 5)
6 (3896, 5)
7 (4383, 5)
8 (4870, 5)
9 (5357, 5)
10 (5844, 5)
11 (6331, 5)
12 (6818, 5)
13 (7305, 5)
14 (7792, 5)
15 (8279, 5)
16 (8766, 5)
17 (9253, 5)
18 (9740, 5)
19 (10227, 5)
20 (10714, 5)
21 (11201, 5)
22 (11688, 5)
23 (12175, 5)
24 (12662, 5)
25 (13149, 5)
26 (13636, 5)
27 (14123, 5)
28 (14610, 5)
29 (15097, 5)
30 (15584, 5)
31 (16071, 5)
32 (16558, 5)
33 (17045, 5)
34 (17532, 5)
35 (18019, 5)
36 (18506, 5)
37 (18993, 5)
38 (19480, 5)
39 (19967, 5)
40 (20454, 5)
41 (20941, 5)
42 (21428, 5)
43 (21915, 5)
44 (22402, 5)
45 (22889, 5)
46 (23376, 5)
47 (23863, 5)
48 (24350, 5)
49 (24837, 5)
50 (25324, 5)
51 (25811, 5)
52 (26298, 5)
53 (26785, 5)
54 (27272, 5)
55 (27759, 5)
56 (28246, 5)
57 (28733, 5)
58 (29220, 5)
59 (29707, 5)


In [64]:
cases_full_melt.shape

(29707, 5)

In [65]:
cases_full_melt = cases_full_melt.reset_index()

In [66]:
cases_full_melt = cases_full_melt.drop("index", axis=1)

In [67]:
cases_full_melt

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,1/22/20,2
1,322,2,1,1/22/20,2
2,322,2,2,1/22/20,0
3,322,2,3,1/22/20,0
4,322,2,4,1/22/20,0
...,...,...,...,...,...
29702,322,4,166,3/22/20,1
29703,322,4,167,3/22/20,1
29704,322,0,168,3/22/20,1
29705,322,2,169,3/22/20,1


In [68]:
cases_full_melt['variable'] = pd.to_datetime(cases_full_melt['variable'],infer_datetime_format=True)

In [69]:
cases_full_melt

,region_codes,continent_codes,country_code,variable,value
0,322,2,0,2020-01-22,2
1,322,2,1,2020-01-22,2
2,322,2,2,2020-01-22,0
3,322,2,3,2020-01-22,0
4,322,2,4,2020-01-22,0
...,...,...,...,...,...
29702,322,4,166,2020-03-22,1
29703,322,4,167,2020-03-22,1
29704,322,0,168,2020-03-22,1
29705,322,2,169,2020-03-22,1


In [70]:
cases_full_melt.to_csv('cases_confirmed.csv')

In [1]:
sources = {
    "confirmed": {
        "url": 
    },
    
}

In [2]:
sources['type']

'confirmed'

In [10]:
sources = {
    "confirmed": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv",
        "local_path": "data/cases_confirmed.csv"
    },
    "recovered": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv",
        "local_path": "data/cases_recovered.csv"    
    },
    "deaths": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv",
        "local_path": "data/cases_deaths.csv"
    },
}

In [11]:
sources['confirmed']['url']

'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'

In [15]:
import pandas as pd

In [22]:
def checkDataChange(data_type):
    old_data = pd.read_csv(sources[data_type]['local_path'])
    old_size = old_data.size
    
    new_data = pd.read_csv(sources[data_type]['url'])
    new_size = new_data.size
    if old_size!=new_size:
        print("Data size mismatch: Old: ",  str(old_size), "; New: ", str(new_size))

In [23]:
checkDataChange('confirmed')

Data size mismatch: Old:  178242 ; New:  33066
